In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor


In [2]:
cf_train=pd.read_csv(r'C:\Users\Rahul kashyap\Downloads\counterfeit_train.csv')
cf_test=pd.read_csv(r'C:\Users\Rahul kashyap\Downloads\counterfeit_test.csv')

In [3]:
cf_train.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402


In [4]:
cf_train.columns

Index(['Medicine_ID', 'Counterfeit_Weight', 'DistArea_ID', 'Active_Since',
       'Medicine_MRP', 'Medicine_Type', 'SidEffect_Level',
       'Availability_rating', 'Area_Type', 'Area_City_Type', 'Area_dist_level',
       'Counterfeit_Sales'],
      dtype='object')

In [5]:
cf_test.columns

Index(['Medicine_ID', 'Counterfeit_Weight', 'DistArea_ID', 'Active_Since',
       'Medicine_MRP', 'Medicine_Type', 'SidEffect_Level',
       'Availability_rating', 'Area_Type', 'Area_City_Type',
       'Area_dist_level'],
      dtype='object')

In [6]:
cf_test['Counterfeit_Sales']=np.nan

In [7]:
cf_train['data']='train'
cf_test['data']='test'
cf_test=cf_test[cf_train.columns]
cf_all=pd.concat([cf_train,cf_test],axis=0)

In [8]:
cf_all.shape

(8523, 13)

In [9]:
cf_all.dtypes

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
data                    object
dtype: object

In [10]:
list(zip(cf_all.columns,cf_all.dtypes,cf_all.nunique()))

[('Medicine_ID', dtype('O'), 1557),
 ('Counterfeit_Weight', dtype('float64'), 415),
 ('DistArea_ID', dtype('O'), 10),
 ('Active_Since', dtype('int64'), 9),
 ('Medicine_MRP', dtype('float64'), 5970),
 ('Medicine_Type', dtype('O'), 16),
 ('SidEffect_Level', dtype('O'), 2),
 ('Availability_rating', dtype('float64'), 7884),
 ('Area_Type', dtype('O'), 4),
 ('Area_City_Type', dtype('O'), 3),
 ('Area_dist_level', dtype('O'), 4),
 ('Counterfeit_Sales', dtype('float64'), 3142),
 ('data', dtype('O'), 2)]

In [11]:
cf_all.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales,data
0,RRA15,13.100,Area046,1995,160.2366,Antimalarial,critical,0.070422,DownTown,Tier 1,Small,1775.5026,train
1,YVV26,NaN,Area027,1983,110.4384,Mstablizers,mild,0.013000,CityLimits,Tier 3,Medium,3069.1520,train
2,LJC15,9.025,Area046,1995,259.4092,Cardiac,mild,0.060783,DownTown,Tier 1,Small,2603.0920,train
3,GWC40,11.800,Area046,1995,99.9830,OralContraceptives,mild,0.065555,DownTown,Tier 1,Small,1101.7130,train
4,QMN13,NaN,Area019,1983,56.4402,Hreplacements,critical,0.248859,MidTownResidential,Tier 1,Small,158.9402,train


In [12]:
# DisArea_ID,Counterfeit_Weight,Medicine_MRP,Availability_rating,Counterfeit_Sales: to numeric
# Area_dist_level,Medicine_ID,Active since : drop
#Area_City_Type,Area_Type : replace with 1,2..
#Medicine_Type,SidEffect_Level : Dummies

In [13]:
cf_all.drop(['Active_Since','Medicine_ID','Area_dist_level'],axis=1,inplace=True)

In [14]:
k=cf_all['DistArea_ID'].str.split("0",expand=True)

In [15]:
k.head()

,0,1,2
0,Area,46,None
1,Area,27,None
2,Area,46,None
3,Area,46,None
4,Area,19,None


In [16]:
cf_all['distAr_ID']=pd.to_numeric(k[1])

In [17]:
cf_all['distAr_ID'].isnull().sum()

0

In [18]:
cf_all.drop(['DistArea_ID'],axis=1,inplace=True)

In [19]:
cf_all['distAr_ID'].head()

0    46
1    27
2    46
3    46
4    19
Name: distAr_ID, dtype: int64

In [20]:
cf_all['SideEffect_level']=pd.get_dummies(cf_all['SidEffect_Level'],drop_first=True,prefix='SideEffect')

In [21]:
del cf_all['SidEffect_Level']
cf_all['SideEffect_level']=cf_all.SideEffect_level.astype(float)

In [22]:
cf_all.head()

,Counterfeit_Weight,Medicine_MRP,Medicine_Type,Availability_rating,Area_Type,Area_City_Type,Counterfeit_Sales,data,distAr_ID,SideEffect_level
0,13.100,160.2366,Antimalarial,0.070422,DownTown,Tier 1,1775.5026,train,46,0.0
1,NaN,110.4384,Mstablizers,0.013000,CityLimits,Tier 3,3069.1520,train,27,1.0
2,9.025,259.4092,Cardiac,0.060783,DownTown,Tier 1,2603.0920,train,46,1.0
3,11.800,99.9830,OralContraceptives,0.065555,DownTown,Tier 1,1101.7130,train,46,1.0
4,NaN,56.4402,Hreplacements,0.248859,MidTownResidential,Tier 1,158.9402,train,19,0.0


In [23]:
cf_all.dtypes

Counterfeit_Weight     float64
Medicine_MRP           float64
Medicine_Type           object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Counterfeit_Sales      float64
data                    object
distAr_ID                int64
SideEffect_level       float64
dtype: object

In [24]:
K=cf_all['Area_City_Type'].str.split(" ",expand=True)

In [25]:
K[1].head()

0    1
1    3
2    1
3    1
4    1
Name: 1, dtype: object

In [26]:
cf_all['City_type']=pd.to_numeric(K[1])

In [27]:
cf_all.drop(['Area_City_Type'],axis=1,inplace=True)

In [28]:
cf_all.head()

,Counterfeit_Weight,Medicine_MRP,Medicine_Type,Availability_rating,Area_Type,Counterfeit_Sales,data,distAr_ID,SideEffect_level,City_type
0,13.100,160.2366,Antimalarial,0.070422,DownTown,1775.5026,train,46,0.0,1
1,NaN,110.4384,Mstablizers,0.013000,CityLimits,3069.1520,train,27,1.0,3
2,9.025,259.4092,Cardiac,0.060783,DownTown,2603.0920,train,46,1.0,1
3,11.800,99.9830,OralContraceptives,0.065555,DownTown,1101.7130,train,46,1.0,1
4,NaN,56.4402,Hreplacements,0.248859,MidTownResidential,158.9402,train,19,0.0,1


In [29]:
print(cf_all['Area_Type'].unique())
print(cf_all['Area_Type'].nunique())

['DownTown' 'CityLimits' 'MidTownResidential' 'Industrial']
4


In [30]:
ArType=pd.get_dummies(cf_all['Area_Type'],drop_first=True,prefix='AreaType')

In [31]:
ArType.head()

,AreaType_DownTown,AreaType_Industrial,AreaType_MidTownResidential
0,1,0,0
1,0,0,0
2,1,0,0
3,1,0,0
4,0,0,1


In [32]:
cf_all=pd.concat([cf_all,ArType],axis=1)
del cf_all['Area_Type']

In [33]:
MedType=pd.get_dummies(cf_all['Medicine_Type'],drop_first=True,prefix='MedType')
cf_all=pd.concat([cf_all,MedType],axis=1)
del cf_all['Medicine_Type']

In [34]:
cf_all.isnull().sum()

Counterfeit_Weight             1463
Medicine_MRP                      0
Availability_rating               0
Counterfeit_Sales              1705
data                              0
distAr_ID                         0
SideEffect_level                  0
City_type                         0
AreaType_DownTown                 0
AreaType_Industrial               0
AreaType_MidTownResidential       0
MedType_Antacids                  0
MedType_Antibiotics               0
MedType_Antifungal                0
MedType_Antimalarial              0
MedType_Antipyretics              0
MedType_Antiseptics               0
MedType_Antiviral                 0
MedType_Cardiac                   0
MedType_Hreplacements             0
MedType_Mstablizers               0
MedType_MuscleRelaxants           0
MedType_OralContraceptives        0
MedType_Statins                   0
MedType_Stimulants                0
MedType_Tranquilizers             0
dtype: int64

In [35]:
#cf_all.loc[cf_all['Counterfeit_Sales'].isnull(),'Counterfeit_Sales']=np.mean(cf_all['Counterfeit_Sales'])

In [36]:
cf_all.loc[cf_all['Counterfeit_Weight'].isnull(),'Counterfeit_Weight']=np.mean(cf_all['Counterfeit_Weight'])

In [37]:
cf_all['Counterfeit_Weight'].isnull().sum()

0

In [38]:
ld_train=cf_all[cf_all['data']=='train']
del ld_train['data']
ld_test=cf_all[cf_all['data']=='test']
ld_test.drop(['Counterfeit_Sales','data'], axis=1,inplace=True)

C:\Users\Rahul kashyap\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [39]:
ld_train.shape, ld_test.shape

((6818, 25), (1705, 24))

In [40]:
cf_train, cf_test = train_test_split(ld_train, test_size=0.2,random_state=2)

In [41]:
cf_train.shape,cf_test.shape

((5454, 25), (1364, 25))

In [42]:
x_train=cf_train.drop('Counterfeit_Sales',axis=1)
y_train=cf_train['Counterfeit_Sales']
x_test=cf_test.drop('Counterfeit_Sales',axis=1)
y_test=cf_test['Counterfeit_Sales']

In [43]:
regressor=RandomForestRegressor()
#n_estimators=100, random_state=0,

In [44]:
param_dist={'n_estimators' : [50,100,500,1000],
            'max_features' : [5,10,15],
            #'min_samples_split' : [25,50,100],
            #'min_samples_leaf': [None,5,10,15],
            #'max_depth': [None,50,150,250,350,450,550]
            'max_depth':[1,2,3,4,5,6]
            }

In [45]:
n_iter_search=10
random_search=RandomizedSearchCV(regressor,param_distributions=param_dist,
                                 n_iter=n_iter_search,
                                 scoring='neg_mean_absolute_error',
                                 cv=5,
                                 verbose = 5,
                                 n_jobs=-1)

In [46]:
search = random_search.fit(x_train,y_train )

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  46 out of  50 | elapsed:   12.5s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   13.4s finished


In [47]:
random_model = search.best_estimator_

In [48]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [49]:
report(search.cv_results_,5)

Model with rank: 1
Mean validation score: -762.536 (std: 23.51823)
Parameters: {'n_estimators': 1000, 'max_features': 15, 'max_depth': 5}

Model with rank: 2
Mean validation score: -763.363 (std: 24.29913)
Parameters: {'n_estimators': 500, 'max_features': 15, 'max_depth': 5}

Model with rank: 3
Mean validation score: -786.787 (std: 20.28224)
Parameters: {'n_estimators': 50, 'max_features': 10, 'max_depth': 5}

Model with rank: 4
Mean validation score: -794.257 (std: 25.51832)
Parameters: {'n_estimators': 500, 'max_features': 15, 'max_depth': 4}

Model with rank: 5
Mean validation score: -836.626 (std: 24.95997)
Parameters: {'n_estimators': 50, 'max_features': 10, 'max_depth': 4}



In [50]:
pred = random_model.predict(x_test)

In [51]:
pred.shape

(1364,)

In [52]:
mean_absolute_error(y_test,pred)

772.4954598839493

In [58]:
test_pred = random_model.predict(ld_test)

In [59]:
ld_test.shape, test_pred.shape

((1705, 24), (1705,))

In [55]:
pd.DataFrame(test_pred).to_csv("Rahul_Kashyap_P3_part2.csv")

In [56]:
#regressor.fit(x_train,y_train)

In [57]:
#v_test=regressor.predict(x_test)